In [13]:
# Imports
import numpy as np
import pandas as pd
import time
import datetime
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Data Preprocessing

In [14]:
# Defines
YEAR = 2017
THRESHOLD = .60
FILL_NA = 0
DATE_CONVERSION = 'timestamps'
PREPROCESSING = 'MeanNormalization'
KFOLD_SPLITS = 10

In [15]:
# Functions
def check_na(train):
    # Finds the number of missing values in each column
    num_of_na = [train.loc[:,col].isnull().sum() for col in train]
    # Divide by rows for proportion 
    prop_na = [num / train.shape[0] for num in num_of_na]
    # Put the proporitons and column names into a df and sort
    na_df = pd.DataFrame({'prop_na' : prop_na, 'column' : train.columns}).sort_values('prop_na')
    return na_df

In [16]:
# Read csvs
if YEAR == 2016:
    properties = pd.read_csv('properties_2016.csv', low_memory = False)
    train = pd.read_csv('train_2016_v2.csv', low_memory = False)
elif YEAR == 2017:
    properties = pd.read_csv('properties_2017.csv', low_memory = False)
    train = pd.read_csv('train_2017.csv', low_memory = False)
# train has Y and properties has features
# Find row intersection of train and properties
train = train.merge(properties, on = 'parcelid', how = 'left')

In [17]:
# Remove all columns above the THRESHOLD
train = train.loc[:, (train.isnull().sum(axis=0) <= (train.shape[0]*THRESHOLD))]

In [18]:
# Replace all NAs with number defined in FILL_NA
# train = train.fillna(FILL_NA)
# Convert transactiondate strings into floats
date_strings = (train.values[:,2])
date_converted = []
if DATE_CONVERSION == 'timestamps':
    for string in date_strings:
        date_converted.append(time.mktime(datetime.datetime.strptime(string, "%Y-%m-%d").timetuple()))
train['transactiondate'] = np.asarray(date_converted)
# Drop the columns with string and int
train = train.drop(columns=['propertycountylandusecode', 'propertyzoningdesc'])
y = train.values[:,1]
y = y.reshape(y.shape[0],1)

In [19]:
# Mean normalization
if PREPROCESSING == "MeanNormalization":
    train = (train - train.mean()) / (train.max() - train.min())
# Fill the missing values
train = train.fillna(FILL_NA)

# Implementation of K-Fold Cross-Validation

In [20]:
# Preprocessing
x = train.values[:,2:]
#if PREPROCESSING == 'MinMax':
    #scaler = MinMaxScaler()
    #scaler.fit(x)
    #x = scaler.transform(x)
# KFolds
train_index_array = []
test_index_array = []
kf = KFold(n_splits = KFOLD_SPLITS, shuffle = True, random_state = 1)
for train_index, test_index in kf.split(x):
    print("TRAIN:", train_index, "TEST:", test_index)
    train_index_array.append(train_index)
    test_index_array.append(test_index)
MY_INDEX = 1
x_train, x_test = x[train_index_array[MY_INDEX]], x[test_index_array[MY_INDEX]]
y_train, y_test = y[train_index_array[MY_INDEX]], y[test_index_array[MY_INDEX]]

TRAIN: [    0     2     3 ... 77609 77611 77612] TEST: [    1    28    30 ... 77602 77607 77610]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   10    24    25 ... 77586 77593 77595]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    3    12    13 ... 77582 77603 77608]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    8    23    40 ... 77567 77597 77599]
TRAIN: [    0     1     3 ... 77610 77611 77612] TEST: [    2    11    22 ... 77569 77574 77605]
TRAIN: [    0     1     2 ... 77609 77610 77611] TEST: [   33    37    39 ... 77585 77601 77612]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [   26    35    36 ... 77587 77588 77591]
TRAIN: [    1     2     3 ... 77609 77610 77612] TEST: [    0     4     6 ... 77600 77606 77611]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    5    41    49 ... 77592 77604 77609]
TRAIN: [    0     1     2 ... 77610 77611 77612] TEST: [    9    15    18 ... 77589 77594 77596]


# Implementation of Neural Network
##### For k-folds, set MY_INDEX variable as the desired fold index in the above cell and re-run the neural network
##### Execution of only one fold is shown

In [21]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(9, input_dim = 27, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'linear')
    ])
sgd = tf.keras.optimizers.SGD(lr=0.1)
model.compile(loss = 'mse', optimizer = sgd)
model.fit(x_train, y_train, epochs = 500, batch_size = 32, verbose = 1)

Epoch 1/500
69851/69851 [==============================] - 2s 35us/step - loss: 0.0293
Epoch 2/500
69851/69851 [==============================] - 2s 30us/step - loss: 0.0284
Epoch 3/500
69851/69851 [==============================] - 2s 33us/step - loss: 0.0284
Epoch 4/500
69851/69851 [==============================] - 3s 38us/step - loss: 0.0284
Epoch 5/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0283
Epoch 6/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0283
Epoch 7/500
69851/69851 [==============================] - 2s 31us/step - loss: 0.0283
Epoch 8/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0283
Epoch 9/500
69851/69851 [==============================] - 2s 29us/step - loss: 0.0283
Epoch 10/500
69851/69851 [==============================] - 2s 31us/step - loss: 0.0283
Epoch 11/500
69851/69851 [==============================] - 2s 30us/step - loss: 0.0283
Epoch 12/500
69851/69851 [===============

69851/69851 [==============================] - 2s 24us/step - loss: 0.0282
Epoch 94/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0282
Epoch 95/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 96/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0282
Epoch 97/500
69851/69851 [==============================] - 3s 40us/step - loss: 0.0282
Epoch 98/500
69851/69851 [==============================] - 3s 37us/step - loss: 0.0282
Epoch 99/500
69851/69851 [==============================] - 2s 33us/step - loss: 0.0282
Epoch 100/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0282
Epoch 101/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0282
Epoch 102/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0282: 1s - loss: 0.026 - ETA
Epoch 103/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0282
Epoch 104/500
698

69851/69851 [==============================] - 2s 27us/step - loss: 0.0281
Epoch 185/500
69851/69851 [==============================] - 2s 25us/step - loss: 0.0281
Epoch 186/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0281
Epoch 187/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 188/500
69851/69851 [==============================] - 2s 30us/step - loss: 0.0281
Epoch 189/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 190/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 191/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0281
Epoch 192/500
69851/69851 [==============================] - 2s 28us/step - loss: 0.0281
Epoch 193/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 194/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0281
Epoch 195/500
69851/69851 [========

69851/69851 [==============================] - 3s 40us/step - loss: 0.0281
Epoch 276/500
69851/69851 [==============================] - 3s 40us/step - loss: 0.0281
Epoch 277/500
69851/69851 [==============================] - 2s 34us/step - loss: 0.0281
Epoch 278/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0281
Epoch 279/500
69851/69851 [==============================] - 3s 39us/step - loss: 0.0281
Epoch 280/500
69851/69851 [==============================] - 2s 29us/step - loss: 0.0281: 0s - loss: 0
Epoch 281/500
69851/69851 [==============================] - 3s 42us/step - loss: 0.0281
Epoch 282/500
69851/69851 [==============================] - 2s 28us/step - loss: 0.0281
Epoch 283/500
69851/69851 [==============================] - 3s 36us/step - loss: 0.0281
Epoch 284/500
69851/69851 [==============================] - 2s 30us/step - loss: 0.0281
Epoch 285/500
69851/69851 [==============================] - 4s 51us/step - loss: 0.0281
Epoch 286/500
69851/6

Epoch 367/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0280:  - ETA: 0s - loss: 
Epoch 368/500
69851/69851 [==============================] - 3s 37us/step - loss: 0.0280
Epoch 369/500
69851/69851 [==============================] - 2s 28us/step - loss: 0.0280
Epoch 370/500
69851/69851 [==============================] - 2s 30us/step - loss: 0.0280
Epoch 371/500
69851/69851 [==============================] - 2s 31us/step - loss: 0.0280
Epoch 372/500
69851/69851 [==============================] - 2s 32us/step - loss: 0.0280
Epoch 373/500
69851/69851 [==============================] - 2s 35us/step - loss: 0.0280
Epoch 374/500
69851/69851 [==============================] - 2s 29us/step - loss: 0.0280
Epoch 375/500
69851/69851 [==============================] - 2s 29us/step - loss: 0.0280
Epoch 376/500
69851/69851 [==============================] - 2s 28us/step - loss: 0.0280
Epoch 377/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0280


69851/69851 [==============================] - 2s 25us/step - loss: 0.0280
Epoch 459/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 460/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0280: 0s
Epoch 461/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 462/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 463/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 464/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 465/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 466/500
69851/69851 [==============================] - 2s 27us/step - loss: 0.0280
Epoch 467/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 468/500
69851/69851 [==============================] - 2s 26us/step - loss: 0.0280
Epoch 469/500
69851/69851 [====

In [43]:
model.evaluate(x_test, y_test, verbose = 1)

7762/7762 [==============================] - 0s 44us/step


0.03756374718656889